<a href="https://colab.research.google.com/github/MJ-7-Kang/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [ ]:
import pandas as pd

## data.csv 파일 읽기
df=pd.read_csv('data.csv',encoding='utf-8',sep=',',header=0)

print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 4-2 JSON 파일 읽기



In [ ]:
import json
import pandas as pd

## data.json 파일 출력
with open ('data.json', mode='r',encoding='utf-8') as f:
    data=json.load(f)
print(data)
## data.json 파일 DataFrame 읽기
df = pd.read_json ('data.json', orient='records', encoding='utf-8')#, lines=false)

print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [ ]:
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()

## 주민등록번호 패턴 생성
pattern = re.compile (r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub (r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open ('callcenter20250301_masked.log.txt', mode='w') as f:
    f.write (masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [ ]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get (url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.03552436828613281, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-09T01:45', 'interval': 900, 'temperature_2m': 19.2}}
서울시 종로구의 현재 온도는 : 19.2°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0   228M      0 --:--:-- --:--:-- --:--:--  229M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 448 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 mesa-vulkan-drivers amd64 23.2.1-1ubuntu3.1~22.04.3 [10.7

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)

## 사이트 접속 대기
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring (page_source)
title_text = tree.xpath ('//title/text()')
print (title_text)
## 드라이버 종료
driver.quit()

ModuleNotFoundError: No module named 'selenium'


# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [31]:
import requests
import pandas as pd

## 데이터 수집 url 및 api key 설정
url= 'http://openapi.seoul.go.kr:8088/7843766f7a6d7975363349635a6b65/json/energyUseDataSummaryInfo/1/5/2025/01/'
api_key = '7843766f7a6d7975363349635a6b65'

year = 2018
month = 3
type = '개인'

params = {
    'serviceKey': api_key,
    'returnType': 'json'
    #'YEAR': "2018",
    #'MON': "03",
    #'MM_TYPE': type
}

API_KEY = "7843766f7a6d7975363349635a6b65"
SERVICE_NAME = "energyUseDataSummaryInfo"  # 회원유형별 통계정보 서비스명
REQUEST_TYPE = "json"
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{REQUEST_TYPE}/{SERVICE_NAME}"
MEMBER_TYPE="개인"

str_year = str(year)
str_month = str(month).zfill(2)

url = f"{BASE_URL}/1/5/{str_year}/{str_month}/"
print (url)

## 데이터 수집
response = requests.get (url, params=params)

## 호출 성공/실패 출력
if response.status_code == 200:
  print("api 호출 성공")
  print(response.json())
else:
  print(f"API 호출 실패: {response.status_code}")


#response['날짜'] = f'{year}{month:02d}'  # 날짜형식: 201501

# DataFrame 생성
data=response.json()['energyUseDataSummaryInfo']['row']
df = pd.DataFrame(data)
# '날짜' 칼럼을 맨 앞으로 재배치
#df.insert(0, '날짜', df.pop('날짜'))
df.head(3)


http://openapi.seoul.go.kr:8088/7843766f7a6d7975363349635a6b65/json/energyUseDataSummaryInfo/1/5/2018/03/
api 호출 성공
{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2018', 'MON': '03', 'MM_TYPE': '소상공인', 'CNT': '26055', 'EUS': '49020790', 'EUS1': '50501786', 'EUS2': '51173059', 'ECO2_1': '-1816632.5', 'ECO2_2': '-770252.18', 'GUS': '3472254', 'GUS1': '3504000', 'GUS2': '3503476', 'GCO2_1': '-31484', 'GCO2_2': '-70524.16', 'WUS': '1105383', 'WUS1': '1075163', 'WUS2': '1069313.6', 'WCO2_1': '33144.7', 'WCO2_2': '11004.0404', 'HUS': '1730.697', 'HUS1': '2100.85', 'HUS2': '2083.087', 'HCO2_1': '-361.2715', 'HCO2_2': '-88.861708', 'REG_DATE': '2018-06-05 04:07:27.0'}, {'YEAR': '2018', 'MON': '03', 'MM_TYPE': '학교', 'CNT': '5504', 'EUS': '139547317', 'EUS1': '140617480', 'EUS2': '137011835', 'ECO2_1': '732659.5', 'ECO2_2': '310647.628', 'GUS': '7375082', 'GUS1': '7414005', 'GUS2': '7285270', 'GCO2_1': '25444.5', 'G

,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WUS1,WUS2,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE
0,2018,03,소상공인,26055,49020790,50501786,51173059,-1816632.5,-770252.18,3472254,...,1075163,1069313.6,33144.7,11004.0404,1730.697,2100.85,2083.087,-361.2715,-88.861708,2018-06-05 04:07:27.0
1,2018,03,학교,5504,139547317,140617480,137011835,732659.5,310647.628,7375082,...,1440488,1533340.2,32372.7,10747.7364,9254.198,11181.43,11466.032,-2069.533,-5297.419296,2018-06-05 04:07:27.0
2,2018,03,종교단체,624,5690732,5798426,5967739,-192350.5,-81556.612,562100,...,38879,38272,-126.5,-41.998,0,0,0,0,0,2018-06-05 04:07:27.0


In [ ]:
import requests
import pandas as pd
import time
from tqdm import tqdm

# 발급받은 인증키를 입력하세요.
API_KEY = "7843766f7a6d7975363349635a6b65"

# API 정보 설정
SERVICE_NAME = "energyUseDataSummaryInfo"  # 회원유형별 통계정보 서비스명
REQUEST_TYPE = "json"
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{REQUEST_TYPE}/{SERVICE_NAME}"

# 수집할 기간 설정
start_year = 2015
end_year = 2024

# 수집된 데이터를 저장할 리스트
all_data = []

# 개인 유형 데이터만 필터링하기 위한 회원유형 코드 (01: 개인)

print("데이터 수집을 시작합니다...")

for year in tqdm(range(start_year, end_year + 1), desc="연도별 데이터 수집"):
    for month in range(1, 13):
        # YYYY, MM 포맷으로 년도와 월을 문자열로 변환
        str_year = str(year)
        str_month = str(month).zfill(2)

        # 요청 파라미터 조합 (회원유형, 년도, 월)
        # 1-1000은 한 번에 가져올 데이터의 범위를 지정합니다. API의 최대 요청 건수에 따라 조절이 필요합니다.
        url = f"{BASE_URL}/1/5/{MEMBER_TYPE}/{str_year}/{str_month}/"

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            data = response.json()

            if SERVICE_NAME in data:
                result = data[SERVICE_NAME]
                if 'row' in result:
                    rows = result['row']
                    for row in rows:
                        all_data.append(row)
                else:
                    # 해당 월에 데이터가 없을 경우
                    print(f"[{str_year}-{str_month}] 데이터가 없습니다.")
            else:
                # API 응답 구조가 예상과 다를 경우
                print(f"[{str_year}-{str_month}] API 응답이 올바르지 않습니다: {data}")

        except requests.exceptions.RequestException as e:
            print(f"[{str_year}-{str_month}] API 호출 중 오류 발생: {e}")
        except Exception as e:
            print(f"[{str_year}-{str_month}] 데이터 처리 중 오류 발생: {e}")

        # API 호출 제한 방지를 위해 1초 대기
        time.sleep(1)

# 수집된 데이터를 데이터프레임으로 변환
if all_data:
    df = pd.DataFrame(all_data)

    # 데이터를 CSV 파일로 저장
    output_filename = f"ecomileage_individual_energy_usage_{start_year}_{end_year}.csv"
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n총 {len(df)}건의 데이터를 수집하여 '{output_filename}' 파일로 저장했습니다.")
else:
    print("\n수집된 데이터가 없습니다. API 키 및 설정 정보를 확인하세요.")


데이터 수집을 시작합니다...


연도별 데이터 수집:   0%|          | 0/10 [00:00<?, ?it/s]

[2015-01] API 응답이 올바르지 않습니다: {'RESULT': {'CODE': 'INFO-200', 'MESSAGE': '해당하는 데이터가 없습니다.'}}
[2015-02] API 응답이 올바르지 않습니다: {'RESULT': {'CODE': 'INFO-200', 'MESSAGE': '해당하는 데이터가 없습니다.'}}
[2015-03] API 응답이 올바르지 않습니다: {'RESULT': {'CODE': 'INFO-200', 'MESSAGE': '해당하는 데이터가 없습니다.'}}
[2015-04] API 응답이 올바르지 않습니다: {'RESULT': {'CODE': 'INFO-200', 'MESSAGE': '해당하는 데이터가 없습니다.'}}


연도별 데이터 수집:   0%|          | 0/10 [00:06<?, ?it/s]


KeyboardInterrupt: 